# 🏥 Medical AI System: Practical Implementation

## Lecture 20 - Capstone Project Practice

### Table of Contents
1. [Environment Setup and Data Loading](#practice-1-environment-setup)
2. [Data Preprocessing Pipeline](#practice-2-data-preprocessing)
3. [Model Training and Evaluation](#practice-3-model-training)
4. [API Development (FastAPI)](#practice-4-api-development)
5. [Model Deployment Simulation](#practice-5-deployment)
6. [Performance Monitoring](#practice-6-monitoring)

---

## Installing and Importing Essential Libraries

### 📚 Required Libraries
- **Data Processing**: numpy, pandas
- **Machine Learning**: scikit-learn, tensorflow/pytorch
- **Visualization**: matplotlib, seaborn
- **API**: fastapi (optional for this practice)
- **Monitoring**: time, logging

In [ ]:
# Install required packages (uncomment if needed)
# !pip install numpy pandas matplotlib seaborn scikit-learn pillow

# Import essential libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score, roc_curve
from sklearn.ensemble import RandomForestClassifier
import time
import warnings
warnings.filterwarnings('ignore')

# Visualization settings
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

print("✅ All libraries loaded successfully!")
print(f"📊 NumPy version: {np.__version__}")
print(f"📊 Pandas version: {pd.__version__}")

---
## Practice 1: Environment Setup and Data Loading

### 🎯 Learning Objectives
- Set up a medical AI development environment
- Load and understand medical dataset structure
- Perform initial data quality checks

### 📖 Key Concepts from Slides
- **Requirements Analysis**: Understanding functional and non-functional requirements
- **Data Collection**: Medical image datasets and metadata

In [ ]:
# 1.1 Generate synthetic medical dataset
def generate_medical_dataset(n_samples=1000, random_state=42):
    """
    Generate a synthetic medical dataset for binary classification
    Simulating medical imaging features and diagnosis labels
    """
    np.random.seed(random_state)
    
    # Generate features (simulating medical imaging measurements)
    age = np.random.randint(20, 80, n_samples)
    feature_1 = np.random.randn(n_samples) * 10 + 50  # e.g., tissue density
    feature_2 = np.random.randn(n_samples) * 5 + 30   # e.g., lesion size
    feature_3 = np.random.randn(n_samples) * 3 + 15   # e.g., contrast ratio
    feature_4 = np.random.randn(n_samples) * 8 + 40   # e.g., texture metric
    
    # Generate labels (0: Normal, 1: Abnormal)
    # Create correlation with features
    risk_score = (age * 0.02 + feature_1 * 0.03 + feature_2 * 0.05 + 
                  feature_3 * 0.04 + feature_4 * 0.02 + np.random.randn(n_samples) * 5)
    labels = (risk_score > np.percentile(risk_score, 70)).astype(int)
    
    # Create DataFrame
    data = pd.DataFrame({
        'patient_id': [f'P{i:04d}' for i in range(n_samples)],
        'age': age,
        'tissue_density': feature_1,
        'lesion_size': feature_2,
        'contrast_ratio': feature_3,
        'texture_metric': feature_4,
        'diagnosis': labels
    })
    
    return data

# Generate dataset
medical_data = generate_medical_dataset(n_samples=1000)

print("🏥 Medical Dataset Generated")
print("=" * 60)
print(f"Total samples: {len(medical_data)}")
print(f"Features: {list(medical_data.columns[1:-1])}")
print(f"\nClass distribution:")
print(medical_data['diagnosis'].value_counts())
print(f"\nFirst 5 records:")
print(medical_data.head())

In [ ]:
# 1.2 Data Quality Validation
def validate_data_quality(df):
    """
    Perform data quality checks
    Following best practices from Data Collection & Processing slide
    """
    print("🔍 Data Quality Validation")
    print("=" * 60)
    
    # Check for missing values
    missing_values = df.isnull().sum()
    print("\n1. Missing Values Check:")
    if missing_values.sum() == 0:
        print("   ✅ No missing values detected")
    else:
        print(f"   ⚠️ Missing values found:\n{missing_values[missing_values > 0]}")
    
    # Check data types
    print("\n2. Data Types:")
    print(df.dtypes)
    
    # Statistical summary
    print("\n3. Statistical Summary:")
    print(df.describe())
    
    # Check for duplicates
    duplicates = df.duplicated().sum()
    print(f"\n4. Duplicate Records: {duplicates}")
    if duplicates == 0:
        print("   ✅ No duplicates found")
    
    print("\n" + "=" * 60)
    print("✅ Data quality validation complete!")

validate_data_quality(medical_data)

---
## Practice 2: Data Preprocessing Pipeline

### 🎯 Learning Objectives
- Implement data preprocessing steps
- Apply feature normalization
- Split data for training and testing

### 📖 Key Concepts from Slides
- **Data Processing**: Image normalization, resizing, data augmentation
- **Data Pipeline Design**: ETL process

In [ ]:
# 2.1 Data Preprocessing Pipeline
def preprocess_medical_data(df):
    """
    Complete preprocessing pipeline for medical data
    """
    print("⚙️ Data Preprocessing Pipeline")
    print("=" * 60)
    
    # Separate features and target
    X = df.drop(['patient_id', 'diagnosis'], axis=1)
    y = df['diagnosis']
    
    print(f"\n1. Feature Selection:")
    print(f"   Features: {list(X.columns)}")
    print(f"   Target: diagnosis (0=Normal, 1=Abnormal)")
    
    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )
    
    print(f"\n2. Train-Test Split:")
    print(f"   Training set: {len(X_train)} samples")
    print(f"   Test set: {len(X_test)} samples")
    print(f"   Train class distribution: {y_train.value_counts().to_dict()}")
    print(f"   Test class distribution: {y_test.value_counts().to_dict()}")
    
    # Feature normalization
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    print(f"\n3. Feature Normalization (StandardScaler):")
    print(f"   Mean: {scaler.mean_}")
    print(f"   Std: {scaler.scale_}")
    
    print("\n" + "=" * 60)
    print("✅ Preprocessing complete!")
    
    return X_train_scaled, X_test_scaled, y_train, y_test, scaler

X_train, X_test, y_train, y_test, scaler = preprocess_medical_data(medical_data)

In [ ]:
# 2.2 Visualize Feature Distributions
def visualize_features(df):
    """
    Visualize feature distributions by diagnosis
    """
    features = ['age', 'tissue_density', 'lesion_size', 'contrast_ratio', 'texture_metric']
    
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    axes = axes.ravel()
    
    for idx, feature in enumerate(features):
        for diagnosis in [0, 1]:
            data = df[df['diagnosis'] == diagnosis][feature]
            axes[idx].hist(data, alpha=0.6, bins=30, 
                          label=f'Diagnosis {diagnosis}',
                          edgecolor='black')
        axes[idx].set_xlabel(feature)
        axes[idx].set_ylabel('Frequency')
        axes[idx].legend()
        axes[idx].set_title(f'Distribution of {feature}')
    
    # Remove extra subplot
    fig.delaxes(axes[5])
    
    plt.tight_layout()
    plt.show()
    
    print("📊 Feature distributions visualized")

visualize_features(medical_data)

---
## Practice 3: Model Training and Evaluation

### 🎯 Learning Objectives
- Train a machine learning model for medical diagnosis
- Evaluate model performance using multiple metrics
- Visualize evaluation results

### 📖 Key Concepts from Slides
- **Model Training Pipeline**: Data loading, model training, validation
- **Evaluation Framework**: Accuracy, Precision, Recall, F1-Score, ROC-AUC

In [ ]:
# 3.1 Model Training
def train_medical_classifier(X_train, y_train):
    """
    Train a Random Forest classifier for medical diagnosis
    """
    print("🤖 Model Training")
    print("=" * 60)
    
    # Initialize model
    model = RandomForestClassifier(
        n_estimators=100,
        max_depth=10,
        random_state=42,
        n_jobs=-1
    )
    
    # Train model with timing
    start_time = time.time()
    model.fit(X_train, y_train)
    training_time = time.time() - start_time
    
    print(f"\nModel: Random Forest Classifier")
    print(f"Parameters:")
    print(f"  - n_estimators: 100")
    print(f"  - max_depth: 10")
    print(f"  - random_state: 42")
    print(f"\nTraining time: {training_time:.4f} seconds")
    
    # Cross-validation
    cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')
    print(f"\nCross-validation scores (5-fold):")
    print(f"  Scores: {cv_scores}")
    print(f"  Mean CV Accuracy: {cv_scores.mean():.4f} (±{cv_scores.std():.4f})")
    
    print("\n" + "=" * 60)
    print("✅ Model training complete!")
    
    return model

model = train_medical_classifier(X_train, y_train)

In [ ]:
# 3.2 Model Evaluation
def evaluate_model(model, X_test, y_test):
    """
    Comprehensive model evaluation following the Evaluation Framework slide
    """
    print("📊 Model Evaluation")
    print("=" * 60)
    
    # Make predictions
    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    
    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    auc_roc = roc_auc_score(y_test, y_pred_proba)
    
    print("\n📈 Performance Metrics:")
    print(f"  Accuracy:  {accuracy:.4f} ({accuracy*100:.2f}%)")
    print(f"  Precision: {precision:.4f} ({precision*100:.2f}%)")
    print(f"  Recall:    {recall:.4f} ({recall*100:.2f}%)")
    print(f"  F1-Score:  {f1:.4f} ({f1*100:.2f}%)")
    print(f"  AUC-ROC:   {auc_roc:.4f} ({auc_roc*100:.2f}%)")
    
    # Confusion Matrix
    cm = confusion_matrix(y_test, y_pred)
    print("\n🔢 Confusion Matrix:")
    print(f"  TN: {cm[0,0]:3d}  |  FP: {cm[0,1]:3d}")
    print(f"  FN: {cm[1,0]:3d}  |  TP: {cm[1,1]:3d}")
    
    print("\n" + "=" * 60)
    
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'auc_roc': auc_roc,
        'confusion_matrix': cm,
        'y_pred': y_pred,
        'y_pred_proba': y_pred_proba
    }

eval_results = evaluate_model(model, X_test, y_test)

In [ ]:
# 3.3 Visualize Evaluation Results
def visualize_evaluation(eval_results, y_test):
    """
    Visualize confusion matrix and ROC curve
    """
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Confusion Matrix Heatmap
    sns.heatmap(eval_results['confusion_matrix'], annot=True, fmt='d', 
                cmap='Blues', ax=axes[0], cbar=False,
                xticklabels=['Normal', 'Abnormal'],
                yticklabels=['Normal', 'Abnormal'])
    axes[0].set_xlabel('Predicted')
    axes[0].set_ylabel('Actual')
    axes[0].set_title('Confusion Matrix')
    
    # ROC Curve
    fpr, tpr, _ = roc_curve(y_test, eval_results['y_pred_proba'])
    axes[1].plot(fpr, tpr, linewidth=2, label=f'ROC (AUC = {eval_results["auc_roc"]:.4f})')
    axes[1].plot([0, 1], [0, 1], 'k--', linewidth=1, label='Random Classifier')
    axes[1].set_xlabel('False Positive Rate')
    axes[1].set_ylabel('True Positive Rate')
    axes[1].set_title('ROC Curve')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print("📊 Evaluation visualizations complete!")

visualize_evaluation(eval_results, y_test)

---
## Practice 4: API Development (Simulation)

### 🎯 Learning Objectives
- Create a prediction function for API deployment
- Simulate API request/response flow
- Implement input validation

### 📖 Key Concepts from Slides
- **API Development**: REST API endpoints (GET /predict, POST /upload)
- **Security**: JWT authentication, Rate limiting, HTTPS encryption

In [ ]:
# 4.1 API Prediction Function
def predict_diagnosis(patient_data, model, scaler):
    """
    Simulate API prediction endpoint
    Input: patient features as dictionary
    Output: prediction result with probability
    """
    try:
        # Validate input
        required_features = ['age', 'tissue_density', 'lesion_size', 
                           'contrast_ratio', 'texture_metric']
        
        for feature in required_features:
            if feature not in patient_data:
                raise ValueError(f"Missing required feature: {feature}")
        
        # Prepare input
        input_data = np.array([[patient_data[f] for f in required_features]])
        
        # Normalize
        input_scaled = scaler.transform(input_data)
        
        # Predict
        prediction = model.predict(input_scaled)[0]
        probability = model.predict_proba(input_scaled)[0]
        
        # Format response
        response = {
            'status': 'success',
            'prediction': int(prediction),
            'diagnosis': 'Abnormal' if prediction == 1 else 'Normal',
            'confidence': float(probability[prediction]),
            'probabilities': {
                'normal': float(probability[0]),
                'abnormal': float(probability[1])
            }
        }
        
        return response
        
    except Exception as e:
        return {
            'status': 'error',
            'message': str(e)
        }

# Test API function
print("🔌 API Prediction Function Test")
print("=" * 60)

# Sample patient data
test_patient = {
    'age': 55,
    'tissue_density': 52.3,
    'lesion_size': 31.5,
    'contrast_ratio': 16.2,
    'texture_metric': 41.8
}

print("\nTest Patient Data:")
for key, value in test_patient.items():
    print(f"  {key}: {value}")

result = predict_diagnosis(test_patient, model, scaler)

print("\nAPI Response:")
import json
print(json.dumps(result, indent=2))
print("\n" + "=" * 60)
print("✅ API simulation complete!")

---
## Practice 5: Model Deployment Simulation

### 🎯 Learning Objectives
- Save and load trained models
- Simulate deployment pipeline
- Test model inference performance

### 📖 Key Concepts from Slides
- **Deployment Pipeline**: CI/CD workflow, Docker build, staging, production
- **Deployment Strategies**: Blue-Green, Canary, Rolling

In [ ]:
# 5.1 Model Serialization
import pickle

def save_model_pipeline(model, scaler, filepath='medical_ai_model.pkl'):
    """
    Save trained model and preprocessing pipeline
    """
    pipeline = {
        'model': model,
        'scaler': scaler,
        'features': ['age', 'tissue_density', 'lesion_size', 'contrast_ratio', 'texture_metric'],
        'version': '1.0.0',
        'trained_date': time.strftime('%Y-%m-%d %H:%M:%S')
    }
    
    with open(filepath, 'wb') as f:
        pickle.dump(pipeline, f)
    
    file_size = os.path.getsize(filepath) / (1024 * 1024)  # Convert to MB
    print(f"✅ Model saved to {filepath} ({file_size:.2f} MB)")
    return filepath

def load_model_pipeline(filepath='medical_ai_model.pkl'):
    """
    Load saved model and preprocessing pipeline
    """
    with open(filepath, 'rb') as f:
        pipeline = pickle.load(f)
    
    print(f"✅ Model loaded from {filepath}")
    print(f"   Version: {pipeline['version']}")
    print(f"   Trained: {pipeline['trained_date']}")
    return pipeline

# Save model
import os
print("💾 Model Deployment - Save Pipeline")
print("=" * 60)
model_path = save_model_pipeline(model, scaler)

# Load model
print("\n📦 Model Deployment - Load Pipeline")
print("=" * 60)
loaded_pipeline = load_model_pipeline(model_path)
print("\n✅ Deployment simulation complete!")

In [ ]:
# 5.2 Inference Performance Testing
def test_inference_performance(model, X_test, n_iterations=100):
    """
    Test model inference speed and latency
    """
    print("⚡ Inference Performance Testing")
    print("=" * 60)
    
    # Single prediction latency
    sample = X_test[0:1]
    latencies = []
    
    for _ in range(n_iterations):
        start = time.time()
        _ = model.predict(sample)
        latency = (time.time() - start) * 1000  # Convert to ms
        latencies.append(latency)
    
    print(f"\nSingle Prediction Latency (n={n_iterations}):")
    print(f"  Mean: {np.mean(latencies):.4f} ms")
    print(f"  Std:  {np.std(latencies):.4f} ms")
    print(f"  Min:  {np.min(latencies):.4f} ms")
    print(f"  Max:  {np.max(latencies):.4f} ms")
    print(f"  95th percentile: {np.percentile(latencies, 95):.4f} ms")
    
    # Batch prediction throughput
    batch_sizes = [1, 10, 100, 200]
    print(f"\nBatch Prediction Throughput:")
    
    for batch_size in batch_sizes:
        batch = X_test[:batch_size]
        start = time.time()
        _ = model.predict(batch)
        duration = time.time() - start
        throughput = batch_size / duration
        print(f"  Batch size {batch_size:3d}: {throughput:8.2f} predictions/sec")
    
    print("\n" + "=" * 60)
    print("✅ Performance testing complete!")
    print("\n💡 Target: Response time < 5s (typically ~100-500ms achieved)")

test_inference_performance(model, X_test)

---
## Practice 6: Performance Monitoring

### 🎯 Learning Objectives
- Implement performance monitoring dashboard
- Track key metrics over time
- Visualize system health

### 📖 Key Concepts from Slides
- **Performance Monitoring**: System health, response time, error rate, active users
- **Monitoring Tools**: Prometheus, Grafana, ELK Stack

In [ ]:
# 6.1 Simulate Performance Metrics
def simulate_monitoring_metrics(duration_minutes=60):
    """
    Simulate performance metrics over time
    """
    np.random.seed(42)
    
    # Generate timestamps
    timestamps = pd.date_range(start='2024-01-01', periods=duration_minutes, freq='1min')
    
    # Simulate metrics
    metrics = pd.DataFrame({
        'timestamp': timestamps,
        'response_time_ms': np.random.normal(200, 50, duration_minutes).clip(50, 500),
        'error_rate': np.random.uniform(0, 2, duration_minutes),
        'active_users': np.random.randint(800, 1500, duration_minutes),
        'cpu_usage': np.random.uniform(20, 80, duration_minutes),
        'memory_usage': np.random.uniform(30, 70, duration_minutes)
    })
    
    return metrics

# Generate monitoring data
monitoring_data = simulate_monitoring_metrics(60)

print("📊 Performance Monitoring Dashboard")
print("=" * 60)
print(f"\nMonitoring Period: {monitoring_data['timestamp'].min()} to {monitoring_data['timestamp'].max()}")
print(f"\nCurrent System Metrics:")
print(f"  Average Response Time: {monitoring_data['response_time_ms'].mean():.2f} ms")
print(f"  Average Error Rate: {monitoring_data['error_rate'].mean():.2f}%")
print(f"  Average Active Users: {monitoring_data['active_users'].mean():.0f}")
print(f"  CPU Usage: {monitoring_data['cpu_usage'].mean():.2f}%")
print(f"  Memory Usage: {monitoring_data['memory_usage'].mean():.2f}%")
print(f"\nSystem Health: {'✅ Healthy' if monitoring_data['error_rate'].mean() < 1 else '⚠️ Warning'}")

In [ ]:
# 6.2 Visualize Performance Metrics
def visualize_monitoring_dashboard(metrics_df):
    """
    Create a comprehensive monitoring dashboard
    """
    fig, axes = plt.subplots(2, 2, figsize=(16, 10))
    
    # Response Time
    axes[0, 0].plot(metrics_df['timestamp'], metrics_df['response_time_ms'], 
                    linewidth=1.5, color='#1E64C8')
    axes[0, 0].axhline(y=metrics_df['response_time_ms'].mean(), 
                       color='red', linestyle='--', linewidth=1, label='Average')
    axes[0, 0].set_xlabel('Time')
    axes[0, 0].set_ylabel('Response Time (ms)')
    axes[0, 0].set_title('Response Time Over Time')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    
    # Error Rate
    axes[0, 1].plot(metrics_df['timestamp'], metrics_df['error_rate'], 
                    linewidth=1.5, color='#FF5252')
    axes[0, 1].axhline(y=1.0, color='orange', linestyle='--', 
                       linewidth=1, label='Threshold (1%)')
    axes[0, 1].set_xlabel('Time')
    axes[0, 1].set_ylabel('Error Rate (%)')
    axes[0, 1].set_title('Error Rate Over Time')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)
    
    # Active Users
    axes[1, 0].plot(metrics_df['timestamp'], metrics_df['active_users'], 
                    linewidth=1.5, color='#4CAF50')
    axes[1, 0].fill_between(metrics_df['timestamp'], metrics_df['active_users'], 
                            alpha=0.3, color='#4CAF50')
    axes[1, 0].set_xlabel('Time')
    axes[1, 0].set_ylabel('Active Users')
    axes[1, 0].set_title('Active Users Over Time')
    axes[1, 0].grid(True, alpha=0.3)
    
    # Resource Usage
    axes[1, 1].plot(metrics_df['timestamp'], metrics_df['cpu_usage'], 
                    linewidth=1.5, label='CPU Usage (%)', color='#FF9800')
    axes[1, 1].plot(metrics_df['timestamp'], metrics_df['memory_usage'], 
                    linewidth=1.5, label='Memory Usage (%)', color='#9C27B0')
    axes[1, 1].set_xlabel('Time')
    axes[1, 1].set_ylabel('Usage (%)')
    axes[1, 1].set_title('System Resource Usage')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)
    
    # Rotate x-axis labels
    for ax in axes.flat:
        ax.tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.show()
    
    print("\n📊 Monitoring dashboard visualized!")

visualize_monitoring_dashboard(monitoring_data)

---
## 🎯 Practice Complete!

### Summary of What We Learned:

#### 1. **System Design & Planning** ✅
- Requirements analysis and validation
- Data quality checks
- Preprocessing pipeline design

#### 2. **Implementation** ✅
- Model training with Random Forest
- Comprehensive evaluation metrics
- API function implementation

#### 3. **Deployment & Validation** ✅
- Model serialization and loading
- Performance testing (latency & throughput)
- Monitoring dashboard simulation

### Key Achievements:

| Metric | Target | Achieved |
|--------|--------|----------|
| Accuracy | > 90% | ✅ Check your results |
| Response Time | < 5s | ✅ ~200-500ms |
| Error Rate | < 1% | ✅ ~0.5% |
| System Uptime | > 99% | ✅ Monitored |

### Next Steps:

1. **Advanced Features**:
   - Deep learning models (CNN for medical images)
   - Transfer learning with pre-trained models
   - Ensemble methods

2. **Production Deployment**:
   - Docker containerization
   - Kubernetes orchestration
   - CI/CD pipeline setup

3. **Clinical Validation**:
   - Real medical dataset testing
   - Expert physician comparison
   - Regulatory compliance (FDA/CE)

4. **User Interface**:
   - React.js frontend development
   - Real-time prediction visualization
   - User feedback integration

### 🎓 Congratulations!

You've successfully completed a hands-on implementation of a medical AI system, covering the entire pipeline from data preprocessing to deployment and monitoring!

---

### 📚 Additional Resources:

- **Documentation**: scikit-learn, TensorFlow, PyTorch
- **Medical AI**: NIH Medical Imaging datasets
- **Deployment**: FastAPI, Docker, Kubernetes documentation
- **Monitoring**: Prometheus, Grafana tutorials

---

**Author**: Lecture 20 - Capstone Project  
**Contact**: homin.park@ghent.ac.kr, powersimmani@gmail.com  
**Course**: Introduction to Biomedical Datascience